In [1]:
import os

In [2]:
%pwd

'd:\\Project1\\Kidney_Disease_Classification\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\Project1\\Kidney_Disease_Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [5]:
from Kidney_Disease_Classification.constants import *
from Kidney_Disease_Classification.utils.common import read_yaml, create_directories
import tensorflow as tf

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    
    def get_training_config(self) -> TrainingConfig:
        training =self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])
        
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        
        return training_config

In [7]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    
    def train_valid_generated(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset= "validation",
            shuffle= False,
            **dataflow_kwargs
        )
        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
        
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.train_generator.samples // self.train_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
        )
        
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [8]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generated()
    training.train()
except Exception as e:
    raise e

[2023-12-30 13:12:41,648: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-30 13:12:41,672: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-30 13:12:41,677: INFO: common: Created directory at: artifacts]
[2023-12-30 13:12:41,687: INFO: common: Created directory at: artifacts\training]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/10
23/23 [==============================] - 114s 5s/step - loss: 12.0257 - accuracy: 0.5702 - val_loss: 6.9566 - val_accuracy: 0.5161
Epoch 2/10
23/23 [==============================] - 88s 4s/step - loss: 3.8915 - accuracy: 0.7584
Epoch 3/10
23/23 [==============================] - 91s 4s/step - loss: 7.9563 - accuracy: 0.6629
Epoch 4/10
23/23 [==============================] - 95s 4s/step - loss: 5.6375 - accuracy: 0.7473
Epoch 5/10
23/23 [==============================] - 89s 4s/step - loss: 3.1142 - accuracy: 0.8062
Epoch 6/10
23/23 [==============================] - 86s